In [1]:
from rich import inspect, print

In [2]:
import csv
from sympy.physics.continuum_mechanics.beam import Beam
from sympy.core.symbol import Symbol
from sympy import symbols

import ipytest
ipytest.autoconfig()

# 🧩 Workbook 05

This week we are augmenting the file format again. It now looks like this:

<pre>
<b>Beam name</b>
Length, [E, I]
P, pin support locations (comma separated)
F, fixed support locations (comma separated)
Load Magnitude, Load Start, Load Order, [Load End]
Load Magnitude, Load Start, Load Order, [Load End]

<b>Beam name</b>
Length, [E, I]
P, pin support locations (comma separated)
F, fixed support locations (comma separated)
Load Magnitude, Load Start, Load Order, [Load End]
Load Magnitude, Load Start, Load Order, [Load End]

...etc.
</pre>
Here is an example of what the format looks like with real data:

```
RoofBeam1
168,3850,42
P,0,168
120,0,0
240,0,0
              # Note the blank line between beam data remains
SteelBeam34
142,47000,4200
P,0,120
F,0
120,0,0
240,0,0
```

Everything is the same as last week. The only change is that there is now a **beam name** added for which we are going to use a new data type: `dict` instead of `list`. 

With `list` we referenced each beam in the file with its index. By using `dict` we can reference each beam _by its name_!

### A note about code re-use

As you have noticed in this course, there has been opportunity for code re-use from previous workbooks. I encourage you to continue review your past workbooks and re-use code when you can!

# Task A

Design a function called `read_beams_file` that takes one parameter, a `str` called `filename`, and returns a list of list of string (`list[list[str]]`) representing the lines of data in the file. 

As you are working on your function, be sure to constantly try calling your function to see what the output is, just like you did with running your cells in the previous workbooks. Always print, print, print so you can see what your code is doing!

Write a test for `read_beams_file` and use two `assert` statements to verify that at least two separate lines in the file match the data you expect.

## Task A response

In [3]:
def read_beams_file(filename: str) -> list[list[str]]:
    """
    Reads the file at 'filename' which is assumed to be a type of multi-table CSV file
    with tables separated by blank lines.
    Returns the data in the rows as a list of list of strings.
    """
    file_data = []
    with open(filename, 'r') as file:
        csv_file = csv.reader(file)
        for line in csv_file:
            file_data.append(line)
    return file_data

read_beams_file('beams.txt')

[['RB1'],
 ['4800', '19200', '1000000'],
 ['P', '3600'],
 ['F', '0'],
 ['100', '2000', '0'],
 [],
 ['FB45'],
 ['228', '28000', '8000'],
 ['P', '63', '100', '200'],
 ['150', '0', '0'],
 ['350', '0', '0'],
 [],
 ['ConcBeam1'],
 ['9000', '35000', '1000000000'],
 ['F', '1000'],
 ['P', '9000'],
 ['12', '0', '0'],
 ['24', '0', '0']]

## Task A tests

In [4]:
def test_read_beams_file():
    file_data = read_beams_file('beams.txt')
    assert file_data[0] == ["RB1"]
    assert file_data[1] == ['4800', '19200', '1000000']
    assert file_data[2] == ['P', '3600']
    assert file_data[3] == ['F', '0']
    assert file_data[4] == ['100', '2000', '0']
    assert file_data[5] == []
    
ipytest.run()

.                                                                                            [100%]
======================================== warnings summary =========================================
..\..\..\Anaconda3\lib\site-packages\pyreadline\py3k_compat.py:8
  C:\Users\cferster\Anaconda3\lib\site-packages\pyreadline\py3k_compat.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
    return isinstance(x, collections.Callable)

-- Docs: https://docs.pytest.org/en/stable/warnings.html
1 passed, 1 warning in 0.03s


# Task B

Design a function called `convert_to_int` that takes one parameter, a `list[list[str]]`, and returns a `list[list[str, int]` (which means a list of lists that can contain a combination of `str` and `int`).

The function is to convert all numeric strings into integers but leave all non-numeric strings as strings.

Write a test for `convert_to_int` that will test the following input scenarios (each have their own `assert` line):
1. A line with both a str and an int
2. A line with all str
3. A line with postive and negative integers

## Task B response

In [5]:
def convert_to_int(file_data: list[list[str]]) -> list[list[str, int]]:
    """
    Returns a list representing 'file_data' but with any of its string elements that
    are numeric being converted to integers.
    """
    converted_data = []
    for line in file_data:
        converted_line = []
        for item in line:
            if item.replace("-","").isnumeric():
                converted_line.append(int(item))
            else:
                converted_line.append(item)
        converted_data.append(converted_line)
    return converted_data

convert_to_int(read_beams_file('beams.txt'))

[['RB1'],
 [4800, 19200, 1000000],
 ['P', 3600],
 ['F', 0],
 [100, 2000, 0],
 [],
 ['FB45'],
 [228, 28000, 8000],
 ['P', 63, 100, 200],
 [150, 0, 0],
 [350, 0, 0],
 [],
 ['ConcBeam1'],
 [9000, 35000, 1000000000],
 ['F', 1000],
 ['P', 9000],
 [12, 0, 0],
 [24, 0, 0]]

## Task B tests

In [6]:
def test_convert_to_int():
    lines_to_test_a = [["cat", "42"]]
    lines_to_test_b = [["cat", "hat"]]
    lines_to_test_c = [["15", "-100"]]
    
    assert convert_to_int(lines_to_test_a) == [["cat", 42]]
    assert convert_to_int(lines_to_test_b) == [["cat", "hat"]]
    assert convert_to_int(lines_to_test_c) == [[15, -100]]
    
ipytest.run()

..                                                                                           [100%]
2 passed in 0.02s


# Task C

Design a function called `separate_beam_data` that takes one parameter, a `list[list[str, int]]` and returns a `dict[str, list[list[str, int]]]` (a dictionary with `str` keys and values of `list[list[str, int]]`).

Similar to **Task C** in **Workbook 04**, this function will separate the beam data. However, it will be different than the code in **Task 04/C** in that you will now "parse" the name of the beam. The name of the beam will become the dictionary key and, with the resulting dictionary, you will be able to see the beam data by using the beam name as the index.

e.g.

**Task 04/C** resulted in this:
```python
beams[1] # Access list index 1 to return the list[list[str, int]]
```

```
[[228, 28000, 756], ['P', 63, 100, 200], [150, 0, 0], [350, 0, 0]]
```

**Task 05/C** will result in this:
```python
beams["FB45"]
```

```
[[228, 28000, 756], ['P', 63, 100, 200], [150, 0, 0], [350, 0, 0]]
```

## Task C response

In [7]:
def separate_beams_data(beams_data: list[list[str, int]]) -> dict[str, list[list[str, int]]]:
    """
    Returns a dict representing the lines of 'beams_data' separated out into individual beams.
    The dict has the beam name as the keys and the list of beam data as values.
    """
    beams = {}
    new_beam_coming = False
    current_beam = ""
    for idx, line in enumerate(beams_data):
        if idx == 0:
            current_beam = line[0]
            beams.update({current_beam: []})
        elif not line:
            new_beam_coming = True
        elif new_beam_coming:
            current_beam = line[0]
            beams.update({current_beam: []})
            new_beam_coming = False
        else:
            beams[current_beam].append(line)
    return beams

## Task C tests

In [8]:
def test_separate_beams_data():
    test_data = [
        ["Beam 1"], ["some", 0, 1, 2], ["data", 5, 6, 7], 
        [], 
        ["Beam 2"], ["more", 12, 13, 14], ["data", 23, 24, 25],
    ]
    assert separate_beams_data(test_data)["Beam 1"] == [["some", 0, 1, 2], ["data", 5, 6, 7]]
    assert separate_beams_data(test_data)["Beam 2"] == [["more", 12, 13, 14], ["data", 23, 24, 25]]
    
ipytest.run()

...                                                                                          [100%]
3 passed in 0.03s


In [9]:
beams = separate_beams_data(convert_to_int(read_beams_file('beams.txt')))

# Task D

Now, this is where things are going to steer away from how they were done in **Workbook 04**. 

In Workbook 04, we had a big loop that added all of the data for one beam into a new `Beam` object for analysis. In **Workbook 05** we are going to break this whole loop up into three separate functions.

First, you are going to design a function called `add_pin_support_to_beam`. It will take **three** parameters, as follows:
1. A sympy `Beam` object
2. An integer representing which support number it is (e.g. `0`, `1`, `2`, etc.)
3. An integer representing a support location on the beam (e.g. `0` or `1800` or whatever)

It will return a `Tuple[Beam, Symbol]` which is a way of saying that it will return **two** values (which will automatically be bundled up in a `tuple` by python).

**Note:** For pin supports, be sure to name your reaction symbol something like `"RP_0"` or `"RP_1"` where `"0"` and `"1"` represent your support number.

> ### Why are we returning _two_ values?

If you look back to **Task 04/D**, when we created a pin support, we had to do a few things:
1. Add our reaction `Symbol` (e.g. `RP_1` or whatever) to a list of reaction symbols
2. Add our reactions as "loads" to the `Beam` object
3. Add our boundary conditions at the support location (within the `Beam` object)

Items **2** and **3** are changes made to the `Beam` object and when we return the altered `Beam` object at the end of our function it will have the changed data within it. 

When we _solve_ the beam object with `.solve_for_reaction_loads()` we have to pass all of our reaction symbols to the method in order for them to be solved. This means, after our function runs, we need to have access to the reaction symbol that was created _within_ the function. Remember, using `return` is the way that we can transfer data from inside the function scope to outside the function scope. By returning the symbol created within the function along with the `Beam`, we can add the symbol to an accumulator in our "master loop".

_Note_: The fact that we need to externally pass the reaction symbols to the `Beam` object is, to me, a design flaw of the sympy `beam` module. I plan on submitting a change to the beam module in the future so that this does not need to happen but, for now, this is the way it has to work. 

# Task D response

In [10]:
def add_pin_support(beam: Beam, support_idx: int, support_loc: int) -> tuple[Beam, Symbol]:
    """
    Returns a 'beam' with a pin support added and the symbol used for the pin support unknown.
    """
    R = symbols(f"RP_{support_idx}")
    beam.apply_load(R, support_loc, -1)
    beam.bc_deflection.append((support_loc, 0))
    return (beam, R)

# Task D test

In [11]:
def test_add_pin_support():
    E, I = symbols("E I")
    test_beam = Beam(10, E, I)
    test_beam_w_support, reaction = add_pin_support(test_beam, 2, 5)
    assert test_beam_w_support.bc_deflection == [(5, 0)]
    
ipytest.run()

....                                                                                         [100%]
4 passed in 0.03s


# Task E

This the second of the three "break out" functions.

Similar to **Task D**, you are now going to design a function called `add_fix_support_to_beam`. It will take **three** parameters, as follows:

1. A sympy `Beam` object
2. An integer representing which support number it is (e.g. `0`, `1`, `2`, etc.)
3. An integer representing a support location on the beam (e.g. `0` or `1800` or whatever)

It will return a `Tuple[Beam, Symbol, Symbol]` which is a way of saying that it will return **three** values: the `Beam` object, the deflection reaction, and the slope reaction.

**Note:** For fix supports, be sure to name your reaction symbols something like `"RD_0"` and `"RD_1"` (for deflection unknowns) and `"RS_0"` and `"RS_1"` (for slope unknowns), where `"0"` and `"1"`, etc. just represent the support number.

## Task E response

In [12]:
def add_fix_support(beam: Beam, support_idx: int, support_loc: int) -> tuple[Beam, Symbol]:
    """
    Returns a 'beam' with a pin support added and the symbol used for the pin support unknown.
    """
    RD = symbols(f"RD_{support_idx}")
    RS = symbols(f"RS_{support_idx}")
    beam.apply_load(RD, support_loc, -1)
    beam.apply_load(RS, support_loc, -2)
    beam.bc_deflection.append((support_loc, 0))
    beam.bc_slope.append((support_loc, 0))
    return (beam, RD, RS)

## Task E tests



In [13]:
def test_add_fix_support():
    E, I = symbols("E I")
    test_beam = Beam(10, E, I)
    test_beam_w_support, rd, rs = add_fix_support(test_beam, 2, 5)
    assert test_beam_w_support.bc_deflection == [(5, 0)]
    assert test_beam_w_support.bc_slope == [(5, 0)]
    
ipytest.run()

.....                                                                                        [100%]
5 passed in 0.05s


# Task F

This is the third of the three "break out" functions.

Design a function called `solve_beam`. It takes one parameter: a `list[list[str]]` which represents all of the data for one beam from the file data. The function returns a `Beam` object with all of the data loaded into it and after `.solve_for_reaction_loads()` has been run on it.

To accomplish this, you will be using the functions you created in **Task D** and **Task E**. Think of them as your "helper" functions.

Feel free to also refer to your **Task 04/D** response for guidance.

**Tests** 

Because we are working with `sympy` objects, doing a proper test on the solved beam is a bit fussier. So I will give you a test to run that checks to see if the reaction load from a UDL comes out as expected and the moment from a cantilevered beam comes out as expected.

Copy-paste the below code into the **...tests** cell


```python
import pytest

def test_solve_beam():
    test_beams = {
        'bm1': [
        [4000, 200e9, 450000000],
        ['P', 0, 4000],
        [100, 0, 0]
        ],
        'bm2': [
        [4000, 200e9, 450000000],
        ['F', 0],
        [100, 0, 0],
        ]     
    }
    bm1 = solve_beam(test_beams['bm1'])
    reactions = bm1.reaction_loads
    reaction_symbols = [symbol for symbol in reactions.keys()]
    pytest.approx(
        float(reactions[reaction_symbols[0]]), 4000*100/2
    )
    bm2 = solve_beam(test_beams['bm2'])
    reactions = bm2.reaction_loads
    reaction_symbols = [symbol for symbol in reactions.keys()]
    pytest.approx(
        float(reactions[reaction_symbols[1]]), 4000*100**2/2
    )
    
ipytest.run()
```
    


## Task F response

In [14]:
def solve_beam(beam_data: list[list[str]]) -> Beam:
    """
    Returns a sympy Beam object populated with the parameters in 'beam_data'.
    """
    supports_acc = []
    for idx, line in enumerate(beam_data):
        if idx == 0: # We know the first line always has L, E, I in it
            L_E_I = line
            beam = Beam(*L_E_I)
        elif 'P' in line: # Pinned supports have one reaction: shear
            for p_idx, support_loc in enumerate(line[1:]): # Skip the "P"
                beam, rp = add_pin_support(beam, p_idx, support_loc)
                supports_acc.append(rp)
        elif 'F' in line: # Fixed supports have two reactions: shear and moment
            for f_idx, support_loc in enumerate(line[1:]): # Skip the "F"
                beam, rd, rs = add_fix_support(beam, f_idx, support_loc)
                supports_acc.append(rd)
                supports_acc.append(rs)
        else:  # Everything else will just be a load
            beam.apply_load(*line)
    beam.solve_for_reaction_loads(*supports_acc)
    return beam
            

## Task F test

In [15]:
import pytest

def test_solve_beam():
    test_beams = {
        'bm1': [
        [4000, 200e9, 450000000],
        ['P', 0, 4000],
        [100, 0, 0]
        ],
        'bm2': [
        [4000, 200e9, 450000000],
        ['F', 0],
        [100, 0, 0],
        ]     
    }
    bm1 = solve_beam(test_beams['bm1'])
    reactions = bm1.reaction_loads
    reaction_symbols = [symbol for symbol in reactions.keys()]
    pytest.approx(
        float(reactions[reaction_symbols[0]]), 4000*100/2
    )
    bm2 = solve_beam(test_beams['bm2'])
    reactions = bm2.reaction_loads
    reaction_symbols = [symbol for symbol in reactions.keys()]
    pytest.approx(
        float(reactions[reaction_symbols[1]]), 4000*100**2/2
    )
    
ipytest.run()

......                                                                                       [100%]
6 passed in 0.20s


# Task G

Try using your functions!

In the cell below, try this:

```python
filename = "beams.txt"
beams_data = separate_beams_data(convert_to_int(read_beams_file(filename)))
print([key for key in beams_data.keys()])
beam = solve_beam(beams_data['RB1'])
beam.plot_shear_force()
beam.plot_bending_moment()
beam.plot_deflection()
```

1. Try putting in the different beam names to see the different diagrams
2. The above code is still a bit script-y. You could probably write some more functions to make things easier to use. For example, using your existing functions, it would probably be easy to write a short function that prints all of the beam names from the beam file. Do you see how you could do it? Don't write it, just think about it and imagine what you would do.
3. Maybe it would be easy to write a function to plot all three diagrams given a solved beam as an input parameter? Don't write it, just think about it and imagine what you would do.
4. Maybe you could write one function that reads all of the beams in a beams file and prints all of the results in one go. To do all this, you would only need one input parameter: the filename! Talk about automation, yeah?
5. You have written a lot of Python code, friend! Lets see what it looks like. Copy and paste all of your functions (not your tests), sequentially (A -> B -> C..., etc.) into the cell below. Stand back and just look at them all. This is what writing a program looks like :)


## Task G response: Paste your functions down below

In [16]:
def read_beams_file(filename: str) -> list[list[str]]:
    """
    Reads the file at 'filename' which is assumed to be a type of multi-table CSV file
    with tables separated by blank lines.
    Returns the data in the rows as a list of list of strings.
    """
    file_data = []
    with open(filename, 'r') as file:
        csv_file = csv.reader(file)
        for line in csv_file:
            file_data.append(line)
    return file_data


def convert_to_int(file_data: list[list[str]]) -> list[list[str, int]]:
    """
    Returns a list representing 'file_data' but with any of its string elements that
    are numeric being converted to integers.
    """
    converted_data = []
    for line in file_data:
        converted_line = []
        for item in line:
            if item.replace("-","").isnumeric():
                converted_line.append(int(item))
            else:
                converted_line.append(item)
        converted_data.append(converted_line)
    return converted_data


def separate_beams_data(beams_data: list[list[str, int]]) -> dict[str, list[list[str, int]]]:
    """
    Returns a dict representing the lines of 'beams_data' separated out into individual beams.
    The dict has the beam name as the keys and the list of beam data as values.
    """
    beams = {}
    new_beam_coming = False
    current_beam = ""
    for idx, line in enumerate(beams_data):
        if idx == 0:
            current_beam = line[0]
            beams.update({current_beam: []})
        elif not line:
            new_beam_coming = True
        elif new_beam_coming:
            current_beam = line[0]
            beams.update({current_beam: []})
            new_beam_coming = False
        else:
            beams[current_beam].append(line)
    return beams


def add_pin_support(beam: Beam, support_idx: int, support_loc: int) -> tuple[Beam, Symbol]:
    """
    Returns a 'beam' with a pin support added and the symbol used for the pin support unknown.
    """
    R = symbols(f"RP_{support_idx}")
    beam.apply_load(R, support_loc, -1)
    beam.bc_deflection.append((support_loc, 0))
    return (beam, R)


def add_fix_support(beam: Beam, support_idx: int, support_loc: int) -> tuple[Beam, Symbol]:
    """
    Returns a 'beam' with a pin support added and the symbol used for the pin support unknown.
    """
    RD = symbols(f"RD_{support_idx}")
    RS = symbols(f"RS_{support_idx}")
    beam.apply_load(RD, support_loc, -1)
    beam.apply_load(RS, support_loc, -2)
    beam.bc_deflection.append((support_loc, 0))
    beam.bc_slope.append((support_loc, 0))
    return (beam, RD, RS)


def solve_beam(beam_data: list[list[str]]) -> Beam:
    """
    Returns a sympy Beam object populated with the parameters in 'beam_data'.
    """
    supports_acc = []
    for idx, line in enumerate(beam_data):
        if idx == 0: # We know the first line always has L, E, I in it
            L_E_I = line
            beam = Beam(*L_E_I)
        elif 'P' in line: # Pinned supports have one reaction: shear
            for p_idx, support_loc in enumerate(line[1:]): # Skip the "P"
                beam, rp = add_pin_support(beam, p_idx, support_loc)
                supports_acc.append(rp)
        elif 'F' in line: # Fixed supports have two reactions: shear and moment
            for f_idx, support_loc in enumerate(line[1:]): # Skip the "F"
                beam, rd, rs = add_fix_support(beam, f_idx, support_loc)
                supports_acc.append(rd)
                supports_acc.append(rs)
        else:  # Everything else will just be a load
            beam.apply_load(*line)
    beam.solve_for_reaction_loads(*supports_acc)
    return beam

# 🎷 Submit Workbook 05

This was a big assignment. Hopefully it went a little faster because you were able to copy-paste and generally re-use code you have written before.

The process of writing functions and "helper" functions is part of what's called "functional program design". You write functions that perform all of the individual small tasks you need to do. From there, you combine them to perform the larger tasks. Eventually, you may end up with a program that just calls one or two functions to do all of the work by calling all of your smaller functions. 

This is one very good way of designing programs. Done properly (using one-task-per-function and writing tests for each function) you can build powerful and robust programs very quickly.

For now, email your completed workbook to me at `cferster@rjc.ca` with the subject line `Workbook 05 Submission`